In [18]:
import pandas as pd
from math import sin, cos, atan2,sqrt ,radians

In [19]:
# Load data
df = pd.read_csv("/home/vagabond/Documents/Goals/SmartChainX-Supply-Chain-Optimizer/02_Route_Optimization/locations.csv")


In [20]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # km

In [21]:
dist_matrix = []

In [22]:
for i in range(len(df)):
    row = []
    for j in range(len(df)):
        dist = haversine(df.iloc[i]['latitude'], df.iloc[i]['longitude'],
                         df.iloc[j]['latitude'], df.iloc[j]['longitude'])
        row.append(int(dist * 1000))  # meters
    dist_matrix.append(row)

In [23]:
import networkx as nx


In [24]:
G = nx.complete_graph(len(df))


In [25]:
G

In [26]:
for i in G.nodes():
    for j in G.nodes():
        if i != j:
            dist = haversine(df.iloc[i]['latitude'], df.iloc[i]['longitude'],
                             df.iloc[j]['latitude'], df.iloc[j]['longitude'])
            G[i][j]['weight'] = dist


In [27]:
from networkx.algorithms.approximation import traveling_salesman_problem
route = traveling_salesman_problem(G, cycle=True)

In [28]:
print("🛣️ Optimized Route (location IDs):", route)
print("➡️ Route Names:", df.loc[route]['location_name'].tolist())

🛣️ Optimized Route (location IDs): [0, 7, 2, 3, 1, 5, 6, 4, 0]
➡️ Route Names: ['Warehouse (Andheri)', 'Chembur', 'Dadar', 'Colaba', 'Bandra', 'Goregaon', 'Mulund', 'Powai', 'Warehouse (Andheri)']


In [29]:
import folium

In [30]:
m = folium.Map(location=[19.07, 72.88], zoom_start=11)


In [31]:
for i in route:
    loc = df.iloc[i]
    folium.Marker(
        location=[loc['latitude'], loc['longitude']],
        popup=loc['location_name'],
        icon=folium.Icon(color='blue', icon='truck', prefix='fa')
    ).add_to(m)

In [32]:
for i in range(len(route)-1):
    loc1 = df.iloc[route[i]]
    loc2 = df.iloc[route[i+1]]
    folium.PolyLine(
        locations=[
            [loc1['latitude'], loc1['longitude']],
            [loc2['latitude'], loc2['longitude']]
        ],
        color="green",
        weight=3,
        opacity=0.8
    ).add_to(m)


In [33]:
m.save("mumbai_optimized_route.html")
print("✅ Map saved as: mumbai_optimized_route.html")

✅ Map saved as: mumbai_optimized_route.html


In [34]:
import streamlit as st

In [35]:
st.set_page_config(page_title="🚛 Mumbai Route Optimizer", layout="wide")
st.title("📍 SmartChainX – Mumbai Route Optimizer")

2025-07-12 02:11:37.142 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 02:11:37.144 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 02:11:37.274 
  command:

    streamlit run /nix/store/h3gsj70b725p1k6qpblm42xwzkn4cri2-python3-3.13.4-env/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-07-12 02:11:37.275 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [36]:


# Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Upload location CSV
uploaded_file = st.file_uploader("📤 Upload Mumbai Locations CSV", type=["csv"])
if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.dataframe(df)

    # Build Graph
    G = nx.complete_graph(len(df))
    for i in G.nodes():
        for j in G.nodes():
            if i != j:
                dist = haversine(df.iloc[i]['latitude'], df.iloc[i]['longitude'],
                                 df.iloc[j]['latitude'], df.iloc[j]['longitude'])
                G[i][j]['weight'] = dist

    # Solve TSP
    route = traveling_salesman_problem(G, cycle=True, method='greedy')
    route_names = df.loc[route]['location_name'].tolist()

    st.subheader("📈 Optimized Route Order")
    for idx, name in enumerate(route_names):
        st.write(f"{idx+1}. {name}")

    # Draw Map
    m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)
    for i in route:
        loc = df.iloc[i]
        folium.Marker(
            location=[loc['latitude'], loc['longitude']],
            popup=loc['location_name'],
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(m)

    for i in range(len(route) - 1):
        loc1 = df.iloc[route[i]]
        loc2 = df.iloc[route[i+1]]
        folium.PolyLine(
            locations=[[loc1['latitude'], loc1['longitude']],
                       [loc2['latitude'], loc2['longitude']]],
            color="green", weight=3, opacity=0.8
        ).add_to(m)

    # Save & Show Map
    m.save("mumbai_optimized_route.html")
    st.subheader("🗺️ Optimized Route Map:")
    st.components.v1.html(m._repr_html_(), height=500)

    # Download link
    with open("mumbai_optimized_route.html", "rb") as f:
        st.download_button("⬇️ Download Map as HTML", data=f, file_name="mumbai_optimized_route.html", mime="text/html")


2025-07-12 02:12:13.512 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 02:12:13.515 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 02:12:13.516 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 02:12:13.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 02:12:13.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
